In [3]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd


import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize


import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [45]:
#Class definitions
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                POVM_normalized_9d[i+2].append(part[i])
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel


In [54]:
# creating five lists POVMs
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

# #print and compare
# print(POVM_unnormalized_5d[0])	## normalized 5-lists POVM vectors
# print(POVM_normalized_5d[0])	## normalized 5-lists POVM vectors

delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
delta_five_new = [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.3812500430281667+0j), (0.40000000000000013-3.0923901048776238e-18j), (0.4+1.898287466587076e-18j), (0.38333333333333347-7.881938927040631e-19j), (0.4312500430281667+3.469446951953614e-18j)]


In [52]:
"""#rechecking validity of delta_five

#inner products of the five-lists with themselves
inner_products = []
for i in range(len(POVM_normalized_5d)):
    inner_products.append(np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]))

inner_products

"""

'#rechecking validity of delta_five\n\n#inner products of the five-lists with themselves\ninner_products = []\nfor i in range(len(POVM_normalized_5d)):\n    inner_products.append(np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]))\n\ninner_products\n\n'

In [55]:

def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    # global delta_five
    # delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    global delta_five_new
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_new

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        np.conj(C63)*y3, np.conj(C63) + np.conj(C64)*y4, np.conj(C63) + np.conj(C64) + np.conj(C65)*y5, np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9,
        # Continue with C73 to C79
        
        # Derivatives with respect to C73 to C79
        np.conj(C73)*y3, np.conj(C73) + np.conj(C74)*y4, np.conj(C73) + np.conj(C74) + np.conj(C75)*y5, np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9,

        # Derivatives with respect to C83 to C89
        np.conj(C83)*y3, np.conj(C83) + np.conj(C84)*y4, np.conj(C83) + np.conj(C84) + np.conj(C85)*y5, np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9,
        # Derivatives with respect to C93 to C99
        np.conj(C93)*y3, np.conj(C93) + np.conj(C94)*y4, np.conj(C93) + np.conj(C94) + np.conj(C95)*y5, np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9,
        #Now the normalization equations
        abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2 + delta_33 - 1,  abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2 + delta_44 - 1,
        abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2 + delta_55 - 1,  abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2 + delta_66 - 1,
        abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2 + delta_77 - 1,  abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2 + delta_88 - 1,
        abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2 + delta_99 - 1
    ]

    assert len(equations) == 35
    return equations

# Example Usage:
initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

Solution to the system: [ 1.70301442e-02 -1.65503253e-02  1.72941925e-02 -1.70301441e-02
 -1.70301444e-02  1.72650540e-02  1.65807201e-02  1.70159935e-02
 -1.65365732e-02  1.72798224e-02 -1.70159934e-02 -1.70159937e-02
  1.72507081e-02  1.65669428e-02  7.74032312e-01 -7.52224196e-01
  7.86033501e-01 -7.74032312e-01 -7.74032311e-01  7.84709124e-01
  7.53605664e-01  1.71583997e-02 -1.66749672e-02  1.74244367e-02
 -1.71583997e-02 -1.71583995e-02  1.73950783e-02  1.67055910e-02
 -2.01948309e-23  1.02899151e+00 -2.77445120e-02  1.04367945e+00
  4.36794494e-02  9.43308780e-01 -5.90310284e-02]
Residuals: [-3.43920882570396e-25, 5.755812493291046e-15, 1.6929047681919773e-12, -7.637189491926932e-12, -1.8652059931290044e-11, -1.9964724318199956e-12, 1.0670062706649719e-11, -3.4363511116463975e-25, 5.738465258531278e-15, 1.6849369122162727e-12, -7.597734941189316e-12, -1.8555970453769388e-11, -1.985300812634705e-12, 1.0610234262367624e-11, -1.5631451668554647e-23, -2.4424906541753444e-15, -2.2725

In [ ]:
# Example Usage:
for i in range(10):
    np.random.seed(i)
    initial_guess = np.concatenate(([0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), np.random.rand(7)))

    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)                   # residuals and residues sum
    residuals_sum = np.sum(np.abs(residuals))

    if residuals_sum < 1e-13:
        print("Seed:", i)
        print("Residuals sum:", residuals_sum)
        print("Solution to the system:", solution)

# print("Solution to the system:", solution)
# print("Residuals:", residuals)
# print("Residuals sum:", residuals_sum)

Working. Now, conducting numerous trials using random initial points and filtering out the best outcomes. Then, converting those results into class objects for analysis.

In [ ]:
seed_res_list = []
for i in range(1500):
    np.random.seed(i)
    initial_guess = np.random.rand(35)  # random initial guess
    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)
    residuals_sum = np.sum(np.abs(residuals))
    if residuals_sum < 1e-11:
        seed_res_list.append((residuals_sum, i))
    seed_res_list.sort()            # sorted in order of increasing residue sum

seed_list = [i[1] for i in seed_res_list]   # create seed list for later use in class objects
sol_list = [seed_sol(i) for i in seed_list]  # storing class objects, list containing all good solutions as class instances

In [ ]:
# doing some clustering to know if the points are close by or far especially the lambda points


points = []
for i in range(len(sol_list)):
    lambdas_point = sorted(abs(sol_list[i].solution()[-7:]))
    points.append(lambdas_point)

data = points  
# Using the Elbow Method to find the optimal number of clusters
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
	kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
	kmeans.fit(data)
	wcss.append(kmeans.inertia_)

# Plotting the results of the Elbow Method
plt.figure(figsize=(10, 7))
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')  # Within cluster sum of squares
plt.show()

# Choose the number of clusters based on the elbow point from the plot
# For this example, let's assume the optimal number is 3 (this should be chosen based on the plot)
optimal_clusters = 5
kmeans_optimal = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
kmeans_optimal.fit(data)

# Reducing dimensions for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(data)

# Plotting the results
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_optimal.labels_, cmap='viridis', edgecolor='k', s=100)
plt.title("Visualization of 9D data in 2D using PCA")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label='Cluster Label')
plt.show()

# Printing the number of centers
print(f"Number of centers (clusters) found: {optimal_clusters}")

# After fitting the KMeans model to the data
labels = kmeans_optimal.labels_

# Count the number of points in each cluster
cluster_counts = np.bincount(labels)

# Print the counts
for i, count in enumerate(cluster_counts):
	print(f"Cluster {i}: {count} points")



In [ ]:
#working way to extend the POVM vectors to 9D, normalization works too
""""
#using nice seed = 1254
sol_1254 = seed_sol(1254).solution          # the sloution we need to add to the 5D POVM vectors
solution = sol_1254

POVM_unnormalized_9d_2 = [list(i) for i in POVM_normalized_5d]         # deep copy 

for i in range(2):
    POVM_unnormalized_9d_2[i].extend([0] *4)

for part in solution_add:
    for i in range(7):
        POVM_unnormalized_9d_2[i+2].append(part[i])

# print(POVM_unnormalized_9d_2[2])
# print(POVM_unnormalized_9d_2[3])
#seems ok

POVM_9d_2 = np.array(POVM_unnormalized_9d_2)   # normalizing the 9D POVM vectors by dividing by sqrt(6), see notes

print(POVM_9d_2[0])
print(POVM_9d_2[1])
print(POVM_9d_2[2])

# sum of abs squared of the 9D POVM vectors
sums_2 = [np.sum(np.abs(i)**2) for i in POVM_9d_2]
print(sums_2) """"

In [56]:
print(seed_sol(1254).solution, seed_sol(1254).res )  # checking the solution and residue for seed 1254, it is good

[ 2.99985484e-01 -2.91533487e-01 -3.04636688e-01  2.99985484e-01
  2.99985484e-01 -3.04123410e-01 -2.92068892e-01  3.26178983e-01
 -3.16988993e-01 -3.31236312e-01  3.26178983e-01  3.26178983e-01
 -3.30678216e-01 -3.17571147e-01  4.78386813e-01 -4.64908415e-01
 -4.85804088e-01  4.78386813e-01  4.78386813e-01 -4.84985563e-01
 -4.65762225e-01  4.18045498e-01 -4.06267198e-01 -4.24527195e-01
  4.18045498e-01  4.18045498e-01 -4.23811915e-01 -4.07013312e-01
  3.29325384e-28  1.02899151e+00  2.77445120e-02  9.87330081e-01
 -1.26699189e-02  9.98891455e-01 -1.15429795e-03] 1.5776144103944575e-15


In [57]:
povm_1254 = seed_sol(1254).full_POVM()  # checking the full POVM for seed 1254. it is a list of 9 9D vectors, normalized (hopefully)

#checking the inner products
relations_1254 = POVM_relations(povm_1254)    # class object for checking the inner products. has all method, and dot_product method.

In [58]:
print(povm_1254[3],'\n' ,povm_1254[4]  )

# inner product
print(abs(relations_1254.dot_product(2,3)))  

[0j, (-0.20412414523193148+0.35355339059327384j), (0.20412414523193162+0.3535533905932737j), (0.20412414523193154+0j), (-0.15811388300841903-0.18257418583505533j), -0.29153348735392065, -0.3169889931662129, -0.4649084152002779, -0.406267197919503] 
 [(-0.4082482904638631+0j), 0j, (-0.20412414523193162-0.3535533905932737j), (-0.10206207261596577-0.17677681699862874j), (-0.07905694150420949+0j), -0.3046366881250902, -0.3312363115700344, -0.4858040878033379, -0.4245271951136413]
0.5132065491623172


In [59]:
rel = relations_1254.all()  # dictionary of all inner products, cleaned up by default
rel

{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 9.936926026465898e-08,
 '16': 0.1443375672974065,
 '17': 0,
 '18': 0.1443375672974065,
 '19': 9.936926027198933e-08,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0.1804219839640731,
 '26': 0,
 '27': 0.1443375672974065,
 '28': 0,
 '29': 0.036084416666666674,
 '33': 0.9999999999999999,
 '34': 0.5132065491623172,
 '35': 0.8101069083609337,
 '36': 0.7147260547463855,
 '37': 0.7147260547463855,
 '38': 0.8087915890668386,
 '39': 0.5166902906428554,
 '44': 1.0,
 '45': 0.42018386597536866,
 '46': 0.8092544935111813,
 '47': 0.6837048860657932,
 '48': 0.7172648364379266,
 '49': 0.6083618168821768,
 '55': 1.0,
 '56': 0.7845103544067316,
 '57': 0.5639429382537904,
 '58': 0.7327973694583723,
 '59': 0.44947345069517197,
 '66': 1.0,
 '67': 0.6414501799308603,
 '68': 0.7844722356848823,
 '69': 0.46353786324629254,
 '77': 0.9999999999999999,
 '78': 0.7838080261733551,
 '79': 0.8099702351879048,
 '88': 1.0,
 '89': 0.421709378287026

In [63]:
((6**.5)*(np.array(povm_1254))).tolist()  # unnormalized 9D POVM vectors

[[0j, (1+0j), (-1+0j), (2+0j), 0j, 0j, 0j, 0j, 0j],
 [(-1+0j), 0j, (1+0j), (0.5+0j), (1.9364916731037087+0j), 0j, 0j, 0j, 0j],
 [(1+0j),
  (-1+0j),
  0j,
  (0.5+0j),
  (0.38729833462074165+0j),
  (0.734811365104068+0j),
  (0.7989720731761296+0j),
  (1.171803590463553+0j),
  (1.0239981588729823+0j)],
 [0j,
  (-0.4999999999999999+0.8660254037844388j),
  (0.5000000000000002+0.8660254037844384j),
  (0.5+0j),
  (-0.3872983346207418-0.4472135954999578j),
  (-0.714108286951238+0j),
  (-0.7764612873358053+0j),
  (-1.1387883943666637+0j),
  (-0.9951473341330859+0j)],
 [(-1+0j),
  0j,
  (-0.5000000000000002-0.8660254037844384j),
  (-0.25-0.43301300000000004j),
  (-0.19364916731037082+0j),
  (-0.7462044428378464+0j),
  (-0.8113599476281321+0j),
  (-1.1899721300764146+0j),
  (-1.0398750099633771+0j)],
 [(1+0j),
  (0.4999999999999999-0.8660254037844388j),
  0j,
  (-0.25+0j),
  (0.5809475019311124+0j),
  (0.7348113651040669+0j),
  (0.7989720731761286+0j),
  (1.1718035904635544+0j),
  (1.023998158872